# 1. 연산자 오버로딩 기본 지식

# 2. 단항 연산자

In [139]:
from array import array
import reprlib
import math
import numbers
import functools
import operator
import itertools


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return (len(self) == len(other) and
                all(a == b for a, b in zip(self, other)))

    def __hash__(self):
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0)

# BEGIN VECTOR_V6_UNARY
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __neg__(self):
        return Vector(-x for x in self)  # <1>

    def __pos__(self):
        return Vector(self)  # <2>
# END VECTOR_V6_UNARY

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))

    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))

    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):  # hyperspherical coordinates
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)],
                                     self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

# BEGIN VECTOR_V6_ADD
    def __add__(self, other):
        print('test')
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a, b in pairs)
        except BaseException:
            print('예외 발생')
            return NotImplemented

#     def __radd__(self, other):
#         return self + other

In [140]:
test = Vector([3, 9, 3])

In [141]:
test + 3

test
예외 발생


TypeError: unsupported operand type(s) for +: 'Vector' and 'int'

In [142]:
test.__add__(3)

test
예외 발생


NotImplemented

In [55]:
test

Vector([3.0, 9.0, 3.0])

In [56]:
test.__neg__

<bound method Vector.__neg__ of Vector([3.0, 9.0, 3.0])>

In [57]:
test

Vector([3.0, 9.0, 3.0])

In [58]:
test.__neg__()

Vector([-3.0, -9.0, -3.0])

In [59]:
test

Vector([3.0, 9.0, 3.0])

In [60]:
for x in test:
    print(x)

3.0
9.0
3.0


In [61]:
[-x for x in test]

[-3.0, -9.0, -3.0]

# 3. 벡터를 더하기 위해 + 오버로딩하기 

In [62]:
itertools.zip_longest

itertools.zip_longest

In [63]:
v1 = Vector([3, 4, 5])

In [64]:
v3 = Vector([1, 1, 1])

In [65]:
pairs = itertools.zip_longest(v1, v3)

In [66]:
pairs

In [67]:
dir(pairs)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [68]:
pairs.__next__()

(3.0, 1.0)

In [69]:
for a, b in pairs:
    print(a, b)

4.0 1.0
5.0 1.0


In [70]:
v1 + [1, 2, 3]

Vector([4.0, 6.0, 8.0])

In [71]:
v1 + [1, 2, 9]

Vector([4.0, 6.0, 14.0])

In [72]:
[1, 2, 9] + v1

TypeError: can only concatenate list (not "Vector") to list

# 4. 벡터를 스칼라와 곱하기 위해 * 오버로딩하기

In [150]:
from array import array
import reprlib
import math
import numbers # 1
import functools
import operator
import itertools


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return (len(self) == len(other) and
                all(a == b for a, b in zip(self, other)))

    def __hash__(self):
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __neg__(self):
        return Vector(-x for x in self)

    def __pos__(self):
        return Vector(self)

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))

    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))

    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):  # hyperspherical coordinates
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)],
                                     self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a, b in pairs)
        except TypeError:
            return NotImplemented

    def __radd__(self, other):
        return self + other

    def __mul__(self, scalar): # 2
        if isinstance(scalar, numbers.Real): # 숫자인지 검사
            return Vector(n * scalar for n in self)
        else: # 3
            return NotImplemented

    def __rmul__(self, scalar):
        return self * scalar # 4

In [116]:
v1 = Vector([1, 2, 3])

In [121]:
v1 * 10

Vector([10.0, 20.0, 30.0])

In [122]:
v1 * True

Vector([1.0, 2.0, 3.0])

In [123]:
v1 * False

Vector([0.0, 0.0, 0.0])

In [124]:
v1 * True

Vector([1.0, 2.0, 3.0])

In [143]:
numbers.Real.mro()

[numbers.Real, numbers.Complex, numbers.Number, object]

In [144]:
numbers.Number.mro()

[numbers.Number, object]

In [146]:
int.mro()

[int, object]

In [147]:
numbers.Integral.mro()

[numbers.Integral,
 numbers.Rational,
 numbers.Real,
 numbers.Complex,
 numbers.Number,
 object]

# 6. 복합 할당 연산자

In [ ]:
Add